In [1]:
import re
import nltk
import pandas as pd
import numpy as np
pd.options.display.max_colwidth = 100 #Çıktıların boyutunu ayarlamak için.

In [2]:
df_idf=pd.read_csv("dergi1.csv",error_bad_lines=False,encoding="utf-8", engine='python')
#print("Veri Türleri:\n\n",df_idf.dtypes)
#print("Total Veri Sayısı,Sütunlar=",df_idf.shape)
dataframe=pd.DataFrame(df_idf)
dataframe.columns

x=dataframe["ozet"]
docs=[]
for i in x:
    a=str(i)
    docs.append(a)
docs=np.array(docs)
classes=[]
x=dataframe["anahtar_kelime"]
for i in x:
    a=str(i)
    classes.append(a)
#docs = np.array(docs)
df_docs = pd.DataFrame({'Dokuman': docs, 
                        'Sinif': classes})
#df_docs

In [3]:
#Kendi oluşturduğumuz  stopwords (689 veri)
with open("stopwords.txt","r+",encoding="utf-8") as dosya:
    dizi=dosya.readlines()
    stop_words_list=[""]
    for i in dizi:
        i=i.replace("\n","")
        kontrol=True
        for j in range(len(stop_words_list)):
            if str(stop_words_list[j])==str(i):
                kontrol=False
                break
            else:
                continue
        if kontrol==True:
            stop_words_list.append(str(i))
        else:
            continue
#stop_words_list

In [4]:
#Kod bloğu hazır,bazı satırlar ekledik ve stopwordsumuzu kullandık.
#@title Default title text
WPT = nltk.WordPunctTokenizer()
#stop_word_list = nltk.corpus.stopwords.words('turkish')
stop_words_list
def norm_doc(single_doc):
    # TR: Dokümandan belirlenen özel karakterleri ve sayıları at
    # EN: Remove special characters and numbers
    single_doc = re.sub(" \d+", "", single_doc)
    single_doc=re.sub("(\\d|\\W)+"," ",single_doc)
    single_doc=re.sub("<!--?.*?-->","",single_doc)
    #single_doc=re.sub("(A-Za-z)","",single_doc)
    pattern = r"[{}]".format(",.;") 
    single_doc = re.sub(pattern, "", single_doc) 
    # TR: Dokümanı küçük harflere çevir
    # EN: Convert document to lowercase   
    single_doc = single_doc.strip()
    single_doc = single_doc.lower()
    # TR: Dokümanı token'larına ayır
    # EN: Tokenize documents
    tokens = WPT.tokenize(single_doc)
    # TR: Stop-word listesindeki kelimeler hariç al
    # EN: Filter out the stop-words 
    filtered_tokens = [token for token in tokens if token not in stop_words_list]
    # TR: Dokümanı tekrar oluştur
    # EN: Reconstruct the document
    single_doc = ' '.join(filtered_tokens)
    return single_doc

norm_docs = np.vectorize(norm_doc) #like magic :)
normalized_documents = norm_docs(docs)
#print(normalized_documents)

In [5]:
from sklearn.feature_extraction.text import CountVectorizer
cv=CountVectorizer(min_df=0.1,max_df=0.9,stop_words=stop_words_list)
word_count_vector=cv.fit_transform(docs)
#print(word_count_vector)

C:\Users\ABDULLAH\Anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:300: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['ismi', 'zaman'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


In [6]:
#bi gram ve uni gram için gerçekleştirdim
cv=CountVectorizer(min_df=0.,max_df=0.9,stop_words=stop_words_list,max_features=100000,ngram_range=(2,3))
word_count_vector=cv.fit_transform(normalized_documents)

In [7]:
#totalde anahtar kelimeler
list(cv.vocabulary_.keys())[:10]


['zaruret muhtaç',
 'muhtaç vatandaşlar',
 'vatandaşlar türkiye',
 'türkiye kabul',
 'kabul kişilere',
 'yardım sosyal',
 'sosyal adaleti',
 'adaleti pekiştirici',
 'pekiştirici tedbirler',
 'tedbirler dağılımının']

In [8]:
from sklearn.feature_extraction.text import TfidfTransformer
tfidf_transformer=TfidfTransformer(smooth_idf=True,use_idf=True)
tfidf_transformer.fit(word_count_vector)

TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)

In [9]:
def sort_coo(coo_matrix):
    tuples = zip(coo_matrix.col, coo_matrix.data)
    return sorted(tuples, key=lambda x: (x[1], x[0]), reverse=True)
 
def extract_topn_from_vector(feature_names, sorted_items, topn=10):
    """get the feature names and tf-idf score of top n items"""
    
    #vector öğesinden yalnızca en iyi n öğelerini kullan
    sorted_items = sorted_items[:topn]
 
    score_vals = []
    feature_vals = []
    
    # kelime indeksi ve karşılık gelen tf-idf skoru
    for idx, score in sorted_items:
        
        #özellik adının ve karşılık gelen puanın kaydını tutun
        score_vals.append(round(score, 3))
        feature_vals.append(feature_names[idx])
 
    #create a tuples of feature,score
    #results = zip(feature_vals,score_vals)
    results= {}
    for idx in range(len(feature_vals)):
        results[feature_vals[idx]]=score_vals[idx]
    
    return results
 

In [10]:
# you only needs to do this once, this is a mapping of index to 
feature_names=cv.get_feature_names()
docs_test=df_idf['ozet'].tolist() 
# get the document that we want to extract keywords from
doc=docs_test[1]
 
#generate tf-idf for the given document
tf_idf_vector=tfidf_transformer.transform(cv.transform([doc]))
 
#sort the tf-idf vectors by descending order of scores
sorted_items=sort_coo(tf_idf_vector.tocoo())
 
#extract only the top n; n here is 10
keywords=extract_topn_from_vector(feature_names,sorted_items,5)
 
# now print the results
print("\n=====Doc=====")
print(doc)
print("\n===Keywords===")
for k in keywords:
    print(k,keywords[k])

#1 dergiyi tf_idf ile hesapladım.2 anahtar kelimeden brini buldum.


=====Doc=====
 Bu çalışmanın amacı üniversite öğrencilerinin öz yönetimli öğrenmeye hazırbulunuşluk düzeylerinin çeşitli değişkenler açısından incelenmesidir. Araştırmanın evrenini 20162017 eğitim öğretim yılında Bilecik Şeyh Edebali Üniversitesi’nin Fen Edebiyat Fakültesi (FEF) Mühendislik Fakültesi (MF) ve İktisadi ve İdari Bilimler Fakültesinin (İİBF) çeşitli lisans programlarında öğrenim gören 1. ve 4. sınıf öğrencileri oluşturmaktadır. Çalışmanın örneklemini ise belirtilen evrenden yansız olarak seçilen 570 öğrenci oluşturmaktadır. Araştırmada veri toplama aracı olarak kişisel bilgi formu ile Fisher ve diğerleri (2001) tarafından geliştirilen ve Türkçeye uyarlama çalışması Şahin ve Erden (2009) tarafından gerçekleştirilen “Öz Yönetimli Öğrenmeye Hazırbulunuşluk Ölçeği (ÖYÖHÖ) kullanılmıştır. Verilerin analizinde; aritmetik ortalama standart sapma gibi betimsel analiz teknikleri; iki değişkenin karşılaştırılmasında ttesti üç ve daha fazla değişkenlerin karşılaştırılmasında ise tek